In [ ]:
import pandas as pd
import plotly.express as px
from pathlib import Path
import os


project_root = Path.cwd().parent
data_path = project_root / "data" / "processed"
de_price_file = data_path / "DE_spot_prices.csv"
hu_price_file = data_path / "HU_spot_prices.csv"

# Check if files exist before loading
if not de_price_file.exists() or not hu_price_file.exists():
    print("throw err")
else:
    de_prices = pd.read_csv(de_price_file, index_col=0, parse_dates=True)
    de_prices.columns = ['DE_Price']

    hu_prices = pd.read_csv(hu_price_file, index_col=0, parse_dates=True)
    hu_prices.columns = ['HU_Price']

    print(de_prices.head(3))
    print("\HU:")
    print(hu_prices.head(3))

                           DE_Price
2025-07-07 00:00:00+02:00    118.84
2025-07-07 01:00:00+02:00    107.42
2025-07-07 02:00:00+02:00    101.92
\HU:
                           HU_Price
2025-07-07 00:00:00+02:00    117.99
2025-07-07 01:00:00+02:00    105.64
2025-07-07 02:00:00+02:00    100.98


In [20]:

de_prices.index = de_prices.index.tz_convert('UTC')
hu_prices.index = hu_prices.index.tz_convert('UTC')

merged_prices = de_prices.join(hu_prices, how='outer')
merged_prices['PRice Delta'] = merged_prices['HU_Price'] - merged_prices['DE_Price']
# Calculate the price delta (HU - DE)
merged_prices['Price_Delta_HU_DE'] = merged_prices['HU_Price'] - merged_prices['DE_Price']

print(" merge ok.")
print(merged_prices.head())

 merge ok.
                           DE_Price  HU_Price  PRice Delta  Price_Delta_HU_DE
2025-07-06 22:00:00+00:00    118.84    117.99        -0.85              -0.85
2025-07-06 23:00:00+00:00    107.42    105.64        -1.78              -1.78
2025-07-07 00:00:00+00:00    101.92    100.98        -0.94              -0.94
2025-07-07 01:00:00+00:00     99.12     97.67        -1.45              -1.45
2025-07-07 02:00:00+00:00    101.00     99.34        -1.66              -1.66


In [18]:
# --- 3. Time Series Plot ---
fig_line = px.line(
    merged_prices,
    x=merged_prices.index,
    y=['DE_Price', 'HU_Price'],
    title='DAM Spot Prices: DE_LU vs. HU',
    labels={'value': ' (€/MWh)', 'timestamp': 'Date'},
    template='plotly_white'
)

fig_line.show()

In [ ]:
# --- 4. Price Distribution Comparison (Box Plot) ---
fig_box = px.box(
    merged_prices,
    y=['DE_Price', 'HU_Price'],
    title='{"Sktala z brki box plot"}',
    labels={'value': 'Price (€/MWh)', 'variable': 'Country'},
    points='all' # Show all data points
)

fig_box.show()

In [21]:
# --- 5. Price Delta Plot ---
fig_delta = px.line(
    merged_prices,
    x=merged_prices.index,
    y='Price_Delta_HU_DE',
    title='Price Spread: HU - DE',
    labels={'Price_Delta_HU_DE': 'Price Delta (€/MWh)', 'timestamp': 'Date'},
    template='plotly_white'
)

fig_delta.add_hline(y=0, line_dash="dash", line_color="red")

fig_delta.show()

# --- 5. Price Delta (Spread) Analysis ---
This plot shows the difference between the Hungarian and German spot prices (HU Price - DE Price).
- A **positive** value means the Hungarian price was higher than the German price.
- A **negative** value means the German price was higher.
This is a key indicator for understanding the market dynamics and congestion between the two regions.